In [3]:
import numpy as np
import pandas as pd

In [46]:
# Записать датасет
df = pd.DataFrame({
    'Комнаты': [3, 1, 2, 3, 1, 3, 2, 1, 3, 1, 2],
    'Площадь': [51, 30, 45, 55, 45, 100, 71, 31, 53, 33, 43],
    'Этаж': [3, 1, 2, 1, 3, 3, 2, 2, 5, 3, 5],
    'Центр': [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
    'Спальный1': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    'Спальный2': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
    'Цена': [2200, 1600, 1900, 2000, 4500, 7000, 5000, 1700, 2100, 1500, 2000]
})

X = df[['Комнаты', 'Площадь', 'Этаж', 'Центр', 'Спальный1', 'Спальный2']]
y = df['Цена']

display(df)

,Комнаты,Площадь,Этаж,Центр,Спальный1,Спальный2,Цена
0,3,51,3,0,1,0,2200
1,1,30,1,0,1,0,1600
2,2,45,2,0,1,0,1900
3,3,55,1,0,1,0,2000
4,1,45,3,1,0,0,4500
5,3,100,3,1,0,0,7000
6,2,71,2,1,0,0,5000
7,1,31,2,0,0,1,1700
8,3,53,5,0,0,1,2100
9,1,33,3,0,0,1,1500


### Задание 1
Написать функцию `multiplication_order(A, B, C)`, которая вернёт строку *"(AxB)xC"*, если количество умножений элементов матриц при умножении $(A\times B)\times C$ меньше либо равно количеству умножений, если выполнять их в порядке $A\times (B\times C)$. В противном случае вернуть строку *"Ax(BxC)"*.

In [22]:
def multiplication_order(A, B, C):
    # Получить размерность матриц
    m, n = A.shape
    n, k = B.shape
    k, p = C.shape

    # В соответствии с формулами матричного умножения вычислить количество операций по перемножению строк и столбцов
    # 1. Если (АхВ)хС:
    mult_1 = k * n * m + p * k * m
    # 2. Если Ах(ВхС):
    mult_2 = p * k * n + p * n * m

    # Для получения выходного значения сравнить полученные числа
    return "(AxB)xC" if mult_1 <= mult_2 else "Ax(BxC)"

# Проверить корректность функции
A = np.array([[1, 2]])
B = np.array([
    [2],
    [1]
])
C = np.array([[5]])
print(multiplication_order(A, B, C))
# (AxB)xC

(AxB)xC


### Задание 2
Написать функцию `best_worst(X, y)`, которая вернёт два числа: `max_corr_idx` (номер признака, наиболее коррелирующего с ценой) и `min_corr_idx` (номер признака, наименее коррелирующего с ценой). Учесть, что корреляция имеет знак, а сила корреляции зависит от абсолютного значения — нужно вернуть наибольший и наименьший признаки по абсолютному значению.

In [ ]:
def best_worst(X, y):
    # Вычислить корреляционную матрицу
    corr_matrix = np.corrcoef(X, y, rowvar=False)

    # По условию задачи нас интересует корреляция признаков с целевым
    # Получить элементы последней строки корреляционной матрицы без последнего элемента
    corr_coefs = corr_matrix[-1][:-1] 
    
    # Найти минимальный и максимальный индексы
    min_corr_idx = np.argmin(np.abs(corr_coefs))
    max_corr_idx = np.argmax(np.abs(corr_coefs))

    # Перевести np.int64 в int
    return int(max_corr_idx), int(min_corr_idx)

print(best_worst(X, y))
# (3, 2)

(3, 2)


### Задание 3
Написать функцию `corr_rank(X)`, возвращающую одно число — ранг корреляционной матрицы.

In [53]:
def corr_rank(X):
    # Вычислить корреляционную матрицу
    corr_matrix = np.corrcoef(X, rowvar=False)

    # Получить ранг матрицы
    return np.linalg.matrix_rank(corr_matrix)

print(corr_rank(X))
# 5

5


### Задание 4
Написать функцию `inverse_matrix(A)`, которая вернёт `None`, если матрица необратима (то есть её определитель по абсолютному значению меньше 0.001), либо вернёт обратную матрицу в виде *numpy*-массива.   
**Подсказка:** использовать `np.linalg.inv`.

In [ ]:
def inverse_matrix(A):
    # Проверить, обратима ли матрица А
    # Вернуть None, если детерминант стремится к нулю, иначе обратную матрицу от А
    det = np.linalg.det(A)
    return None if np.abs(det) < 0.001 else np.linalg.inv(A)

A = np.array([
    [1, 2],
    [2, 1]
])
print(inverse_matrix(A))
# array([[-0.33333333, 0.66666667],[ 0.66666667, -0.33333333]])

[[-0.33333333  0.66666667]
 [ 0.66666667 -0.33333333]]


### Задание 5
Написать функцию `fit_model(X, y)`, которая вернёт *numpy*-массив с оптимальными коэффициентами $a$, найденными методом **OLS**:  
$a = (X^T X)^{-1} X^T y$

In [37]:
def fit_model(X, y):
    # Получить вектор весов модели с помощью МНК (OLS)
    w_hat = np.linalg.inv(X.T @ X) @ X.T @ y
    return w_hat.to_numpy()

print(fit_model(X, y))
# [-574.12295766 65.33255763 141.80223878 1566.16246224 12.32450391 -315.34552489]

[-574.12295766   65.33255763  141.80223878 1566.16246224   12.32450391
 -315.34552489]
